# Advanced document indexing

# Splitting and ingesting HTML content

## Splitting and ingesting the content of a single URL (on Cornwall)

### Preparing the Chroma DB collections

In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import getpass

OPENAI_API_KEY = getpass.getpass('Enter your OPENAI_API_KEY')

Enter your OPENAI_API_KEY ········


In [2]:
corwnall_granular_collection = Chroma(
    collection_name="cornwall_granular",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

corwnall_granular_collection.reset_collection() # in case it already exists

In [3]:
corwnall_coarse_collection = Chroma(
    collection_name="cornwall_coarse",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

corwnall_coarse_collection.reset_collection() # in case it already exists

### Loading the HTML content with the AsyncHtmlLoader 

In [4]:
from langchain_community.document_loaders import AsyncHtmlLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
destination_url = "https://en.wikivoyage.org/wiki/Cornwall"

In [6]:
html_loader = AsyncHtmlLoader(destination_url)

In [7]:
docs = html_loader.load()

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  3.30it/s]


In [8]:
len(docs)

1

### Splitting into granular chunks with the HTMLSectionSplitter 

In [9]:
from langchain_text_splitters import HTMLSectionSplitter

In [10]:
headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2")]
html_section_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)

In [11]:
def split_docs_into_granular_chunks(docs):
    all_chunks = []
    for doc in docs:
        html_string = doc.page_content #A
        temp_chunks = html_section_splitter.split_text(html_string) #B
        all_chunks.extend(temp_chunks) 

    return all_chunks

#A Extract the HTML text from the document
#B Each chunk is a H1 or H2 HTML section

In [12]:
granular_chunks = split_docs_into_granular_chunks(docs)

#### Ingesting granular chunks

In [13]:
corwnall_granular_collection.add_documents(documents=granular_chunks)

['b551a340-961f-4535-a8c4-f9aa0308f7c3',
 '82dd4309-9bf4-4a84-989a-9bfeefe64d8e',
 'b3261037-5b56-4c49-9b09-c2e29fe58abd',
 '877baeb1-00ac-4d00-9e7f-f1b35ba5964f',
 '7449f64e-342b-427d-9a9e-4ecab8dca083',
 '7d0b6bed-ec7d-48aa-9325-4ce995ffb0c7',
 '4764f8b4-b353-4a94-b7e3-86f5e048aabb',
 'aa5e313c-aea9-4534-a480-f615c455b7eb',
 '782dd672-bf04-4369-8882-ce0683dac15d',
 '3c2d016a-4d8a-404b-8317-99ad583dee2a',
 'b99f42cb-2bf5-4b34-99ce-d0a9772d88ae',
 'f6ac1b56-4b0d-4e00-9f85-d81c499484c1',
 'fae710bf-ac7e-411e-87df-54c0967637db',
 '952981fc-ff76-4810-b814-a2744766f695',
 '117c725f-2149-4bb6-8365-145da0482767',
 '424d967e-475a-4a1c-81ef-e345503ab51b',
 '9903dc1c-3c40-4cdd-a351-46c5dd1139bb',
 '3febe6d4-f309-4f76-ad92-32029e88faf7',
 '34243c27-18f6-495d-8c22-cd8a84001c03']

#### Searching granular chunks

In [14]:
results = corwnall_granular_collection.similarity_search(query="Events or festivals in Cornwall",k=3)
for doc in results:
    print(doc)

page_content='Cornwall' metadata={'Header 1': 'Cornwall'}
page_content='Festivals 
 [ edit ] 
 
 These festivals tend to not be public holidays and not all are celebrated fully across the county. 
   
 AberFest 
 .   A Celtic cultural festival celebrating “All things” Cornish and Breton that takes place biennially (every two years) in Cornwall at Easter. The AberFest Festival alternates with the Breizh – Kernow Festival that is held in Brandivy and Bignan (in Breizh/Bretagne – France) on the alternate years.       ( updated Jun 2023 ) 
 Golowan , sometimes also  Goluan  or  Gol-Jowan  is the Cornish word for the Midsummer celebrations, most popular in the Penwith area and in particular  Penzance  and  Newlyn . The celebrations are conducted from the 23rd of June (St John's Eve) to the 28th of June (St Peter's Eve) each year, St Peter's Eve being the more popular in Cornish fishing communities. The celebrations are centred around the lighting of bonfires and fireworks and the performanc

### Splitting into coarse chunks with the RecursiveCharacterTextSplitter 

In [15]:
from langchain_community.document_transformers import Html2TextTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [16]:
html2text_transformer = Html2TextTransformer()

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000, chunk_overlap=300
)

In [18]:
def split_docs_into_coarse_chunks(docs):
    text_docs = html2text_transformer.transform_documents(docs) #A 
    coarse_chunks = text_splitter.split_documents(text_docs)

    return coarse_chunks
#A transform HTML docs into clean text docs

In [19]:
coarse_chunks = split_docs_into_coarse_chunks(docs)

#### Ingesting coarse chunks

In [20]:
corwnall_coarse_collection.add_documents(documents=coarse_chunks)

['e98191bf-3dcd-42b4-b67b-e510381f3bfa',
 'd507709f-773f-47ca-90f3-4e6b7ed0f0cb',
 'db6c4506-99d1-478f-beb7-452e4c12abf0',
 'b47b7924-2100-4b58-bda6-836f9f276eb4',
 '9d41c131-0e13-41a7-964f-5224138a1459',
 'b9823a8c-cb21-41ec-977a-0d22a14db997',
 '719c01dd-eae7-4216-a908-98892173d83f',
 'c2f2107e-f3ca-4403-b93a-9fab141555ea',
 '3c9ca3a3-b960-4c36-ad4f-c7d152403b2a',
 'c2c48076-0b03-4e38-85b6-2063e67f5997',
 '95fa6e3c-3fc4-40b6-bc35-98b5b1883ea9',
 '4e86e015-8b82-47bf-bb4c-6d85e85a4e4c',
 'c4a53289-821d-4384-a6ca-cc0cb6d2a9a2']

#### Searching coarse chunks

In [21]:
results = corwnall_coarse_collection.similarity_search(query="Festivals in Cornwall",k=3)
for doc in results:
    print(doc)

page_content='### Spirits

[edit]

    _See also:Liquor_

Gin and rum are also produced in Cornwall. A popular brand of Cornish rum is
Dead Man's Fingers which has multiple flavours and is bottled in St. Ives.

## Festivals

[edit]

These festivals tend to not be public holidays and not all are celebrated
fully across the county.

AberFest. A Celtic cultural festival celebrating “All things” Cornish and
Breton that takes place biennially (every two years) in Cornwall at Easter.
The AberFest Festival alternates with the Breizh – Kernow Festival that is
held in Brandivy and Bignan (in Breizh/Bretagne – France) on the alternate
years. (updated Jun 2023)

**Golowan** , sometimes also _Goluan_ or _Gol-Jowan_ is the Cornish word for
the Midsummer celebrations, most popular in the Penwith area and in particular
Penzance and Newlyn. The celebrations are conducted from the 23rd of June (St
John's Eve) to the 28th of June (St Peter's Eve) each year, St Peter's Eve
being the more popular in Corni

## Splitting and ingesting the content of various URLs (across UK destinations)

### Preparing the Chroma DB collections

In [22]:
uk_granular_collection = Chroma(
    collection_name="uk_granular",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

uk_granular_collection.reset_collection() #in case it already exists

In [23]:
uk_coarse_collection = Chroma(
    collection_name="uk_coarse",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

uk_coarse_collection.reset_collection() #in case it already exists

### Splitting and ingesting HTML content with the HTMLSectionSplitter 

In [24]:
uk_destinations = [
    "Cornwall", "North_Cornwall", "South_Cornwall", "West_Cornwall", 
    "Tintagel", "Bodmin", "Wadebridge", "Penzance", "Newquay",
    "St_Ives", "Port_Isaac", "Looe", "Polperro", "Porthleven"
    "East_Sussex", "Brighton", "Battle", "Hastings_(England)", 
    "Rye_(England)", "Seaford", "Ashdown_Forest"
]

wikivoyage_root_url = "https://en.wikivoyage.org/wiki"

In [25]:
uk_destination_urls = [f'{wikivoyage_root_url}/{d}' for d in uk_destinations]

In [26]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    docs =  html_loader.load() #B
    
    for doc in docs:
        print(doc.metadata)
        granular_chunks = split_docs_into_granular_chunks(docs)
        uk_granular_collection.add_documents(documents=granular_chunks)

        coarse_chunks = split_docs_into_coarse_chunks(docs)
        uk_coarse_collection.add_documents(documents=coarse_chunks)
#A Loader for one destination
#B Documents of one destination 

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.55it/s]


{'source': 'https://en.wikivoyage.org/wiki/Cornwall', 'title': 'Cornwall – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  2.16it/s]


{'source': 'https://en.wikivoyage.org/wiki/North_Cornwall', 'title': 'North Cornwall – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.82it/s]


...

{'source': 'https://en.wikivoyage.org/wiki/Rye_(England)', 'title': 'Rye (England) – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.38it/s]


{'source': 'https://en.wikivoyage.org/wiki/Seaford', 'title': 'Seaford – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  2.92it/s]


{'source': 'https://en.wikivoyage.org/wiki/Ashdown_Forest', 'title': 'Ashdown Forest – Travel guide at Wikivoyage', 'language': 'en'}


#### Searching 

In [27]:
granular_results = uk_granular_collection.similarity_search(query="Events or festivals in East Sussex",k=4)
for doc in granular_results:
    print(doc)

page_content='Brighton' metadata={'Header 1': 'Brighton'}
page_content='Seaford' metadata={'Header 1': 'Seaford'}
page_content='Penzance' metadata={'Header 1': 'Penzance'}
page_content='Festivals 
 [ edit ] 
 
 These festivals tend to not be public holidays and not all are celebrated fully across the county. 
   
 AberFest 
 .   A Celtic cultural festival celebrating “All things” Cornish and Breton that takes place biennially (every two years) in Cornwall at Easter. The AberFest Festival alternates with the Breizh – Kernow Festival that is held in Brandivy and Bignan (in Breizh/Bretagne – France) on the alternate years.       ( updated Jun 2023 ) 
 Golowan , sometimes also  Goluan  or  Gol-Jowan  is the Cornish word for the Midsummer celebrations, most popular in the Penwith area and in particular  Penzance  and  Newlyn . The celebrations are conducted from the 23rd of June (St John's Eve) to the 28th of June (St Peter's Eve) each year, St Peter's Eve being the more popular in Cornish 

In [28]:
coarse_results = uk_coarse_collection.similarity_search(query="Events or festivals in East Sussex",k=4)
for doc in coarse_results:
    print(doc)

page_content='### Events

[edit]

A market during the Brighton Festival. The Theatre Royal is the red building.
A colourful parade down Queens Road during Pride in 2016.

  * **Brighton Racecourse** has flat-racing April-Oct. It's on Freshfield Rd a mile east of town centre.
  * **Plumpton Racecourse** is National Hunt (jumps races) Nov-March, but it's 10 mi (16 km) north in Lewes.
  * Brighton Festival Fringe: early May – early June, ☏ +44 1273 764900, info@brightonfringe.org. The Fringe runs at the same time as the main festival, and features over 600 events, including comedy, theatre, music, and "open houses" (local artists exhibiting in their own homes) and tours (haunted pubs, Regency Brighton, churches, cemeteries, sewers, etc.)_(date needs fixing)_
  * Brighton Festival: May, ☏ +44 1273 709709 (tickets), tickets@brightonfestival.org. The Brighton Festival, in May each year, is the second biggest arts festival in Great Britain (coming closely behind Edinburgh). Music of all sorts

In [29]:
granular_results = uk_granular_collection.similarity_search(query="Beaches in Conrwall",k=4)
for doc in granular_results:
    print(doc)

page_content='Cornwall' metadata={'Header 1': 'Cornwall'}
page_content='North Cornwall' metadata={'Header 1': 'North Cornwall'}
page_content='West Cornwall' metadata={'Header 1': 'West Cornwall'}
page_content='South Cornwall' metadata={'Header 1': 'South Cornwall'}


In [30]:
coarse_results = uk_coarse_collection.similarity_search(query="Beaches in Cornwall",k=4)
for doc in coarse_results:
    print(doc)

page_content='## Understand

[edit]

The north coast on the Celtic Sea, part of the Atlantic Ocean, is more exposed
than the south coast, and therefore has a wilder nature. The High Cliff,
between Boscastle and St Gennys, is the highest sheer-drop cliff in Cornwall
at 223 metres (732 ft).

Beaches, which form an important part of the tourist industry, include Bude,
Polzeath, Watergate Bay, Perranporth, Porthtowan, Fistral Beach, Newquay, St
Agnes, St Ives, and on the south coast Gyllyngvase beach in Falmouth and the
large beach at Praa Sands further to the south-west.

## Get in

[edit]

### By train

[edit]

Regular trains run on the main line from London Paddington (8 daily all the
way through Cornwall to Penzance, 5 hr 30 min), Bristol, Birmingham etc. to
Penzance. There is also an overnight sleeper train which runs Su-F nights from
London Paddington and Penzance.

### By car

[edit]

Cornwall can be accessed by road via the A30 which runs from the end of the M5
at Exeter, all the w

# Embedding strategy

## Embedding child chunks with ParentDocumentRetriever

In [88]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Setting up the Parent Document retriever

In [89]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500) #B

child_chunks_collection = Chroma( #C
    collection_name="uk_child_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

child_chunks_collection.reset_collection() #D

doc_store = InMemoryStore() #E

parent_doc_retriever = ParentDocumentRetriever( #F
    vectorstore=child_chunks_collection,
    docstore=doc_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#A Splitter to generate child granular chunks from parent coarse chunks
#B Vector store collection to host child granular chunks
#D Make sure the collection is empty
#E Document store to host parent coarse chunks
#F Retriever to link parent coarse chunks to child granular chunks

### Ingesting the content into doc and vector store

In [90]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    print(f'Ingesting {destination_url}')
    parent_doc_retriever.add_documents(text_docs, ids=None) #D

#A Loader for destination web page
#B HTML documents of one destination 
#C Transform HTML docs into clean text deocs
#D Ingest coarse chunks into document store and granular chunks into vector store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.46it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.86it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  9.77it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  6.56it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.86it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


In [91]:
# list(doc_store.yield_keys()) #A <===IMPORTANT: reenable before pushing code in
#A Show the keys of the added coarse chunks

### Performing a search on granular information 

In [92]:
retrieved_docs = parent_doc_retriever.invoke("Cornwall Ranger")

In [93]:
len(retrieved_docs)

4

In [94]:
retrieved_docs[0]

Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="Trains from London take about 3 hr 20 min to Plymouth.\n\n### By car\n\n[edit]\n\nCornwall can be accessed by road via the A30 which runs from the end of the M5\nat Exeter, all the way through the heart of Devon and Cornwall down to Land's\nEnd. It is a grade-separated expressway as far as Carland Cross near Truro\n(the expressway is expected to be open as far as Camborne (between Redruth and\nHayle) by March 2024). You can also get to Cornwall via the A38, crossing the\nRiver Tamar at Plymouth via the Tamar Bridge, which levies a toll on eastbound\nvehicles. On summer Saturdays and during bank holiday weekends roads to\nCornwall are usually busy.\n\n## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All D

### Comparing with direct semantic search on child chunks

In [95]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [96]:
len(child_docs_only)

4

In [97]:
child_docs_only[0]

Document(metadata={'doc_id': '34645d23-ed05-4a53-b3af-c8ab21e3f513', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [98]:
# IMPORTANT: as you can see a granular search would have identified the chunk, but it would have lost the usefulcontext about travelling in Cornwall

## Embedding child chunks with MultiVectorRetriever

In [78]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import uuid

### Setting up the Multi vector retriever

In [108]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500) #B

child_chunks_collection = Chroma( #C
    collection_name="uk_child_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

child_chunks_collection.reset_collection() #D

doc_byte_store = InMemoryByteStore() #E
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #F
    vectorstore=child_chunks_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Splitter to generate child granular chunks from parent coarse chunks
#C Vector store collection to host child granular chunks
#D Make sure the collection is empty
#E Document store to host parent coarse chunks
#F Retriever to link parent coarse chunks to child granular chunks

### Ingesting the content into doc and vector store

In [109]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    coarse_chunks = parent_splitter.split_documents(text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_granular_chunks = []
    for i, coarse_chunk in enumerate(coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        granular_chunks = child_splitter.split_documents([coarse_chunk]) #F

        for granular_chunk in granular_chunks:
            granular_chunk.metadata[doc_key] = coarse_chunk_id #G

        all_granular_chunks.extend(granular_chunks)

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(all_granular_chunks) #H
    multi_vector_retriever.docstore.mset(list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into parent coarse chunks
#E Iterate over the parent coarse chunks
#F Create child granular chunks form each parent coarse chunk
#G Link each child granular chunk to its parent coarse chunk
#H Ingest the child granular chunks into the vector store
#I Ingest the parent coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.77it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.10it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 13.42it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.86it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.20it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [110]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall Ranger")

In [111]:
len(retrieved_docs)

4

In [112]:
retrieved_docs[0]

Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="Trains from London take about 3 hr 20 min to Plymouth.\n\n### By car\n\n[edit]\n\nCornwall can be accessed by road via the A30 which runs from the end of the M5\nat Exeter, all the way through the heart of Devon and Cornwall down to Land's\nEnd. It is a grade-separated expressway as far as Carland Cross near Truro\n(the expressway is expected to be open as far as Camborne (between Redruth and\nHayle) by March 2024). You can also get to Cornwall via the A38, crossing the\nRiver Tamar at Plymouth via the Tamar Bridge, which levies a toll on eastbound\nvehicles. On summer Saturdays and during bank holiday weekends roads to\nCornwall are usually busy.\n\n## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All D

In [114]:
##IMPORTANT: same as PArent Document retriever, but more control and flexibility on how to link child to parent chunks

### Comparing with direct semantic search on child chunks

In [115]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [117]:
len(child_docs_only)

4

In [118]:
child_docs_only[0]

Document(metadata={'doc_id': '04c7f88e-e090-4057-af5b-ea584e777b3f', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [120]:
## IMPORTANT: Same as before

## Embedding summaries with MultiVectorRetriever

In [127]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import uuid

### Setting up the Multi vector retriever (similar to when embedding child chunks)

In [128]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A

summaries_collection = Chroma( #B
    collection_name="uk_summaries",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

summaries_collection.reset_collection() #C

doc_byte_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=summaries_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host parent coarse chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Setting up the summarization chain

In [129]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

In [130]:
summarization_chain = (
    {"document": lambda x: x.page_content} #A
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{document}") #B
    | llm
    | StrOutputParser()

#A Grab the text content from the document
#B Instantiate a prompt asking to generate summary of the provided text
#C Send the LLM the instantiated prompt 
#D Extract the summary text from the response

### Ingesting the coarse chunks and related summaries into doc and vector store

In [132]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    coarse_chunks = parent_splitter.split_documents(text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_summaries = []
    for i, coarse_chunk in enumerate(coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        summary_text =  summarization_chain.invoke(coarse_chunk) #F
        summary_doc = Document(page_content=summary_text, metadata={doc_key: coarse_chunk_id})

        all_summaries.append(summary_doc)

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(all_summaries) #H
    multi_vector_retriever.docstore.mset(list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into coarse chunks
#E Iterate over the coarse chunks
#F Generate a summary for the coarse chunk thorugh the summarization chain
#G Link each summary to its related coarse chunk
#H Ingest the summaries into the vector store
#I Ingest the coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 10.20it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 10.31it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.23it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 10.61it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.61it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


In [ ]:
# COMMENT: the code above is similar to when ingesting child chunks, but it is slower because of the summarization step
# which invokes the LLM.
# The processing can be speeded up by parallelizing the outer for loop on the destination urls.

### Performing a search on granular information

In [139]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall travel")

In [140]:
len(retrieved_docs)

4

In [149]:
retrieved_docs

[Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="Trains from London take about 3 hr 20 min to Plymouth.\n\n### By car\n\n[edit]\n\nCornwall can be accessed by road via the A30 which runs from the end of the M5\nat Exeter, all the way through the heart of Devon and Cornwall down to Land's\nEnd. It is a grade-separated expressway as far as Carland Cross near Truro\n(the expressway is expected to be open as far as Camborne (between Redruth and\nHayle) by March 2024). You can also get to Cornwall via the A38, crossing the\nRiver Tamar at Plymouth via the Tamar Bridge, which levies a toll on eastbound\nvehicles. On summer Saturdays and during bank holiday weekends roads to\nCornwall are usually busy.\n\n## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All 

### Comparing with direct semantic search on summaries

In [145]:
summary_docs_only =  summaries_collection.similarity_search("Cornwall Travel")

In [146]:
len(summary_docs_only)

4

In [148]:
summary_docs_only

[Document(metadata={'doc_id': 'ee55d250-bc53-46ce-9204-8fd2c1a05662'}, page_content="Cornwall is a county located in the southwest of the United Kingdom, known for its distinctive character, warm climate, and beautiful coastline. It is popular among holidaymakers due to its rich Celtic heritage, cultural tourism, and historical connections to arts and mining, which is recognized by UNESCO. Over 30% of Cornwall is designated as an Area of Outstanding Natural Beauty (AONB). \n\nThe county is divided into three regions: North Cornwall, featuring a rugged coastline and surfing spots; South Cornwall, known for its picturesque English Channel coast; and West Cornwall, which includes the famous Land's End. The content also covers visitor information, transportation options, attractions, local cuisine, and safety tips for travelers."),
 Document(metadata={'doc_id': '9e257b4f-64b8-40a0-935f-dcbf627553a6'}, page_content='Cornwall is a popular tourist destination known for its beaches, including 

In [152]:
# COMMENT: a direct search on summaries retrieves denser information, but it is missing out on useful details. 
# However, you might consider using the summaries directly if after testing they prove adequate.

## Embedding hypotetical questions with MultiVectorRetriever

In [160]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import uuid
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

### Setting up the Multi vector retriever (same as when embedding summaries)

In [161]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A

hypotetical_questions_collection = Chroma( #B
    collection_name="uk_hypotetical_questions",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

hypotetical_questions_collection.reset_collection() #C

doc_byte_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=hypotetical_questions_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host parent coarse chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Setting up the chain to generate hypotetical questions

In [162]:
class HypotheticalQuestions(BaseModel):
    """Generate hypothetical questions for given text."""

    questions: List[str] = Field(..., description="List of hypotetical questions for given text")

In [163]:
llm_with_structured_output = ChatOpenAI(model="gpt-4o-mini", 
        openai_api_key=OPENAI_API_KEY).with_structured_output(
        HypotheticalQuestions
)

In [164]:
hypotetical_questions_chain = (
    {"document_text": lambda x: x.page_content} #A
    | ChatPromptTemplate.from_template( #B
        "Generate a list of exactly 4 hypothetical questions that the below text could be used to answer:\n\n{document_text}"
    )
    | llm_with_structured_output #C
    | (lambda x: x.questions) #D
)

#A Grab the text content from the document
#B Instantiate a prompt asking to generate 4 hypotetical questions on the provided text
#C Invoke the LLM configured to return an object containing the questions as a typed list of strings
#D Grab the list of questions from the response

### Ingesting the coarse chunks and related hypotetical questions into doc and vector store

In [167]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    coarse_chunks = parent_splitter.split_documents(text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_hypotetical_questions = []
    for i, coarse_chunk in enumerate(coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        hypotetical_questions = hypotetical_questions_chain.invoke(coarse_chunk) #F
        hypotetical_questions_docs = [Document(page_content=question, metadata={doc_key: coarse_chunk_id})
                                              for question in hypotetical_questions] #G

        all_hypotetical_questions.extend(hypotetical_questions_docs)

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(all_hypotetical_questions) #H
    multi_vector_retriever.docstore.mset(list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into coarse chunks
#E Iterate over the coarse chunks
#F Generate a list of hypotetical questions for the coarse chunk thorugh the question generation chain
#G Link each hypotetical question to its related coarse chunk
#H Ingest the hypotetical questions into the vector store
#I Ingest the coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.54it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 10.40it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.62it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  4.29it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 10.53it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [169]:
retrieved_docs = multi_vector_retriever.invoke("How can you go to Brighton from London?")

In [170]:
len(retrieved_docs)

2

In [171]:
retrieved_docs

[Document(metadata={'source': 'https://en.wikivoyage.org/wiki/Brighton', 'title': 'Brighton – Travel guide at Wikivoyage', 'language': 'en'}, page_content='### By plane\n\n[edit]\n\nThe city\'s proximity to London means Brighton is well served by airports.\nBrighton can be reached from Gatwick by train in as little as 25 minutes\n£9.80-£11.90, Jan 2023).\n\n  * 50.8332-0.2923 Shoreham Airport (Brighton City Airport), Cecil Pashley Way, Shoreham-by-Sea, BN43 5FF (Probably best to get a train from Brighton to Shoreham \\- about 15 minutes, then a taxi from there to the airport), ☏ +44 1273 467373, reception@flybrighton.com. This airport (**ESH** IATA) is 5 miles (8 km) to the west of Brighton. It is the nearest airport for light aircraft and also offers sightseeing flights. However, there are no scheduled flights from here. This is the oldest licensed airport in the UK. (updated Sep 2017)\n\n## Get around\n\n[edit]\n\n50°50′14″N 0°8′56″W\n\nMap of Brighton\n\nBrightonians often give dire

### Inspecting possible questions matching our question through semantic search

In [174]:
hypotetical_question_docs_only =  hypotetical_questions_collection.similarity_search("How can you go to Brighton from London?")

In [175]:
len(hypotetical_question_docs_only)

4

In [176]:
hypotetical_question_docs_only

[Document(metadata={'doc_id': 'af848894-8591-4c28-8295-f3b833ffaa43'}, page_content='What if someone wanted to travel from Brighton to London quickly?'),
 Document(metadata={'doc_id': '7fa14e56-270c-4461-88ab-9b546afb07b1'}, page_content='What if someone wants to attend a performance at Glyndebourne Opera House, how can they arrange their visit from Brighton?'),
 Document(metadata={'doc_id': '7fa14e56-270c-4461-88ab-9b546afb07b1'}, page_content='If a traveler is looking for a day trip from Brighton to France, what options do they have?'),
 Document(metadata={'doc_id': '7fa14e56-270c-4461-88ab-9b546afb07b1'}, page_content='How might a visitor plan their itinerary to include both Lewes and Worthing in one day from Brighton?')]

# Granular chunk expansion with MultiVectorRetriever

In [190]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import uuid

### Setting up the Multi vector retriever

In [191]:
granular_chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=500) #A

granular_chunks_collection = Chroma( #B
    collection_name="uk_granular_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

granular_chunks_collection.reset_collection() #C

expanded_chunk_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=granular_chunks_collection,
    byte_store=expanded_chunk_store
)
#A Splitter to generategranular chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host expanded chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Ingesting granular and expanded chunks into doc and vector store

In [192]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    granular_chunks = granular_chunk_splitter.split_documents(text_docs) #D

    expanded_chunk_store_items = []
    for i, granular_chunk in enumerate(granular_chunks): #E

        this_chunk_num = i #F
        previous_chunk_num = i-1 #F
        next_chunk_num = i+1 #F
        
        if i==0: #F
            previous_chunk_num = None
        elif i==(len(granular_chunks)-1): #F
            next_chunk_num = None

        expanded_chunk_text = "" #G
        if previous_chunk_num: #G
            expanded_chunk_text += granular_chunks[previous_chunk_num].page_content
            expanded_chunk_text += "\n"

        expanded_chunk_text += granular_chunks[this_chunk_num].page_content #G
        expanded_chunk_text += "\n"

        if next_chunk_num: #G
            expanded_chunk_text += granular_chunks[next_chunk_num].page_content
            expanded_chunk_text += "\n"

        expanded_chunk_id = str(uuid.uuid4()) #H
        expanded_chunk_doc = Document(page_content=expanded_chunk_text) #I

        expanded_chunk_store_item = (expanded_chunk_id, expanded_chunk_doc)
        expanded_chunk_store_items.append(expanded_chunk_store_item)

        granular_chunk.metadata[doc_key] = expanded_chunk_id #K
            
    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(granular_chunks) #H
    multi_vector_retriever.docstore.mset(expanded_chunk_store_items) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into granular chunks
#E Iterate over the granular chunks
#F determine the index of the current chunk and its previous and next chunks
#G Assemble the text of the expanded chunk by including the previous and next chunk
#H Generate the ID of the expanded chunk
#I Create the expanded chunk document
#K Link each granular chunk to its related expanded chunk
#H Ingest the granular chunks into the vector store
#I Ingest the expanded chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.35it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.88it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.77it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.77it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.34it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [193]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall Ranger")

In [194]:
len(retrieved_docs)

4

In [195]:
retrieved_docs[0]

Document(page_content="Buses only serve designated stops when in towns; otherwise, you can flag them\ndown anywhere that's safe for them to stop.\n\n### By train\n\n[edit]\n\n**CrossCountry Trains** and **Great Western Railway** operate regular train\nservices between the main centres of population, the latter company also\nserving a number of other towns on branch lines. For train times and fares\nvisit National Rail Enquiries.\nThe **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.\n\nThe **Lost Gardens of

### Comparing with direct semantic search on granular chunks

In [196]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [197]:
len(child_docs_only)

4

In [198]:
child_docs_only[0]

Document(metadata={'doc_id': '04c7f88e-e090-4057-af5b-ea584e777b3f', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [ ]:
# COMMENT: the expanded chunk has more useful context